## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fake_news_classifier/train.csv' )

Mounted at /content/drive


In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.9.2'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

'''Embedding: This is a layer that can be used to create word embeddings (also known as word vectors) from a given input. Word embeddings are a way of representing words in a numerical format that captures their meaning.
pad_sequences: This is a function that can be used to pad sequences to the same length. This is often necessary when working with sequences of varying lengths, as many neural network architectures expect inputs of fixed size.
Sequential: This is a model class that can be used to create simple feedforward neural networks, where the layers are connected sequentially.
one_hot: This function can be used to encode a text into a one-hot encoded representation.
LSTM: Long Short-Term Memory (LSTM) is a type of Recurrent Neural Network (RNN) that is often used for sequential data such as time series or natural language processing.
Dense : Dense layer is a standard layer type that works for most cases. It works by just performing a dot product between the input and a weight matrix, add a bias term and apply an activation function on the result.
'''

'Embedding: This is a layer that can be used to create word embeddings (also known as word vectors) from a given input. Word embeddings are a way of representing words in a numerical format that captures their meaning.\npad_sequences: This is a function that can be used to pad sequences to the same length. This is often necessary when working with sequences of varying lengths, as many neural network architectures expect inputs of fixed size.\nSequential: This is a model class that can be used to create simple feedforward neural networks, where the layers are connected sequentially.\none_hot: This function can be used to encode a text into a one-hot encoded representation.\nLSTM: Long Short-Term Memory (LSTM) is a type of Recurrent Neural Network (RNN) that is often used for sequential data such as time series or natural language processing.\nDense : Dense layer is a standard layer type that works for most cases. It works by just performing a dot product between the input and a weight m

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [20]:
messages.reset_index(inplace=True)

In [21]:
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[81, 1334, 1286, 2571, 4209, 3588, 3752, 594, 2082, 1604],
 [956, 4481, 4968, 4291, 1640, 2507, 999],
 [4864, 1779, 1343, 1614],
 [2508, 4543, 1967, 2465, 4749, 1246],
 [1452, 1640, 4770, 388, 4295, 2499, 1640, 915, 1828, 4554],
 [266,
  1628,
  69,
  1065,
  4824,
  1990,
  3678,
  2450,
  3569,
  306,
  3889,
  1597,
  1212,
  3978,
  999],
 [2341, 284, 2713, 4651, 4946, 4447, 2927, 3482, 99, 2162, 4587],
 [807, 4576, 1225, 4504, 2571, 4679, 1990, 2250, 99, 2162, 4587],
 [101, 1585, 1229, 3318, 4000, 1563, 2058, 1600, 1990, 2011],
 [1541, 2269, 3255, 2785, 3198, 2646, 1887, 2802],
 [4901, 4658, 55, 4593, 2553, 53, 83, 1446, 4989, 238, 2495],
 [2465, 1293, 4209, 1563, 1990, 2571],
 [2360, 2723, 811, 1315, 3573, 2273, 4345, 2324, 2913],
 [1353, 1886, 2249, 1449, 2858, 2884, 3291, 99, 2162, 4587],
 [406, 4122, 2136, 3350, 3606, 99, 2162, 4587],
 [867, 4364, 1556, 2603, 4716, 2037, 53, 217, 2790, 513],
 [1321, 2221, 4481],
 [2236, 50, 3117, 4218, 1990, 4000, 3826, 999],
 [3611, 1121, 49

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr[1]

[956, 4481, 4968, 4291, 1640, 2507, 999]

### Embedding Representation

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[  81 1334 1286 ...    0    0    0]
 [ 956 4481 4968 ...    0    0    0]
 [4864 1779 1343 ...    0    0    0]
 ...
 [ 747 4416  572 ...    0    0    0]
 [1519 2571 4451 ...    0    0    0]
 [2021 1940 4511 ...    0    0    0]]


In [31]:
embedded_docs[1]

array([ 956, 4481, 4968, 4291, 1640, 2507,  999,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [32]:
embedded_docs[0]

array([  81, 1334, 1286, 2571, 4209, 3588, 3752,  594, 2082, 1604,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [33]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [47]:
embedding_vector_features=40 
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 20, 100)           56400     
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 336,901
Trainable params: 336,901
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
len(embedded_docs),y.shape

(18285, (18285,))

In [49]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [50]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [52]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 14ms/step - loss: 0.2913 - accuracy: 0.8610 - val_loss: 0.2000 - val_accuracy: 0.9195
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1377 - accuracy: 0.9489 - val_loss: 0.2227 - val_accuracy: 0.9157
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0973 - accuracy: 0.9651 - val_loss: 0.2289 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0737 - accuracy: 0.9764 - val_loss: 0.2886 - val_accuracy: 0.9064
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0510 - accuracy: 0.9842 - val_loss: 0.3149 - val_accuracy: 0.9077
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0392 - accuracy: 0.9869 - val_loss: 0.3110 - val_accuracy: 0.9051
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0276 - accuracy: 0.9906 - val_loss: 0.3321 - val_accuracy: 0.9117

### Performance Metrics And Accuracy

In [53]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [54]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,y_pred)

array([[3176,  243],
       [ 364, 2252]])

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8994200497100249

In [58]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      3419
           1       0.90      0.86      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.89      0.90      6035
weighted avg       0.90      0.90      0.90      6035

